Домашнее задание
обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
при обучении моделей обязательно использовать кроссвалидацию
вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
сделать выводы о том, какая модель справилась с задачей лучше других
(опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [1]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('./train_case2.csv', sep=';', index_col='id')

In [8]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [15]:
df.describe()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,53.339358,1.349571,164.359229,74.205690,128.841429,96.632414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,6.759594,0.476838,8.210126,14.395757,153.991223,188.471505,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,29.583562,1.000000,55.000000,10.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,48.394521,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,53.980822,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,58.430137,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,64.967123,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
df.loc[df['ap_hi'] < 0, 'ap_hi'] = df['ap_hi'] * (-1)
df.loc[df['ap_lo'] < 0, 'ap_lo'] = df['ap_lo'] * (-1)

In [14]:
df['age'] = df['age'] / 365

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  float64
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(2), int64(10)
memory usage: 9.0 MB


In [17]:
df.columns

Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')

In [18]:
df_train = df.drop(columns='cardio')
df_train

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
id,,,,,,,,,,,
0,50.391781,2,168,62.0,110,80,1,1,0,0,1
1,55.419178,1,156,85.0,140,90,3,1,0,0,1
2,51.663014,1,165,64.0,130,70,3,1,0,0,0
3,48.282192,2,169,82.0,150,100,1,1,0,0,1
4,47.873973,1,156,56.0,100,60,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
99993,52.712329,2,168,76.0,120,80,1,1,1,0,1
99995,61.920548,1,158,126.0,140,90,2,2,0,0,1
99996,52.235616,2,183,105.0,180,90,3,1,0,1,0


In [21]:
df_target = df['cardio']
df_target

id
0        0
1        1
2        1
3        1
4        0
        ..
99993    0
99995    1
99996    1
99998    1
99999    0
Name: cardio, Length: 70000, dtype: int64

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_target, random_state=0)

In [24]:
df_target.value_counts(normalize=True)

0    0.5003
1    0.4997
Name: cardio, dtype: float64

In [29]:
lr = LogisticRegression(solver="liblinear")

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

In [30]:
y_pred

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [66]:
def metrics_model_lr(scor):
    scor = scor
    metrics = cross_val_score(lr, df_train, df_target, scoring=scor, cv = 5)
    metric =  np.mean(metrics)
    return metric

In [67]:
roc_auc1 = metrics_model_lr('roc_auc')
f1_score1 = metrics_model_lr('f1')
precision1 = metrics_model_lr('precision')
recall1 = metrics_model_lr('recall')

In [52]:
from sklearn.tree import DecisionTreeClassifier

In [56]:
tree = DecisionTreeClassifier(random_state=1)

param_grid=[{
             'max_depth': range(1, 7), 
             'min_samples_leaf': range(1, 7), 
            }]

grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)
tree = grid_search.best_estimator_

In [57]:
grid_search.best_params_

{'max_depth': 6, 'min_samples_leaf': 3}

In [68]:
def metrics_model_tree(scor):
    scor = scor
    metrics = cross_val_score(tree, df_train, df_target, scoring=scor, cv = 5)
    metric =  np.mean(metrics)
    return metric

In [69]:
roc_auc2 = metrics_model_tree('roc_auc')
f1_score2 = metrics_model_tree('f1')
precision2 = metrics_model_tree('precision')
recall2 = metrics_model_tree('recall')

In [70]:
table_metrics = pd.DataFrame({'name_metrics': ['roc_auc', 'f1', 'precision', 'recall'], 'metrics_lr': [roc_auc1, f1_score1, precision1, recall1], \
                              'metrics_tree': [roc_auc2, f1_score2, precision2, recall2]})

In [71]:
table_metrics

,name_metrics,metrics_lr,metrics_tree
0,roc_auc,0.784281,0.793803
1,f1,0.707777,0.710867
2,precision,0.741872,0.767512
3,recall,0.676720,0.662254


Определённо лучше справилась модель "дерево решений", но метрики в целом очень близки, так как нет дисбаланса классов. roc_auc_curve лучше подходит при дисбалансе классов, так как отображает  долу верных попаданий от доли ложно-положительных.